In [355]:
import numpy as np
np.random.seed(1)


In [356]:
import math

In [357]:
X = 'input.data'
y1 = 'label.data'


In [358]:
from nnfs.datasets import spiral_data


In [359]:
class Layer_Dense:
    def __init__(self,n_inpt, n_neur):
        self.weights = 0.10 * np.random.randn(n_inpt, n_neur)
        self.biases = np.zeros((1, n_neur))

    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.dot(inputs, self.weights) + self.biases

    def backward(self, dvalues):
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases = np.sum(dvalues, axis = 0, keepdims = True)
        self.dinputs = np.dot(dvalues, self.weights.T)


In [360]:
class Activation_ReLu:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.maximum(0, inputs)
    
    def backward(self, dvalues):
        self.dinputs = dvalues.copy()
        self.dinputs[self.inputs<=0] = 0
        

In [361]:
class Activation_Softmax:
    def forward(self, inputs):
        exp_values = np.exp(inputs- np.max(inputs, axis = 1, keepdims = True))
        probabilities = exp_values / np.sum(exp_values, axis = 1, keepdims = True)
        self.output = probabilities

    def backward(self, dvalues):
        self.dinputs = np.empty_like(dvalues)
        for index, (single_output, single_dvalues) in enumerate(zip(self.output, dvalues)):
            single_output = single_output.reshape(-1, 1)
            jacobian_matrix = np.diagflat(single_output) - np.outer(single_output, single_output)
            self.dinputs[index] = np.dot(jacobian_matrix, single_dvalues)



In [362]:
class LossFunction:
    def calculate(self, output, y):
        samples_losses = self.forward(output, y)
        data_loss = np.mean(samples_losses)
        return data_loss

class CrossEntropy(LossFunction): 
    def forward(self, y_pred, y_true):
        samples = len(y_pred)
        y_pred_clipped = np.clip(y_pred, 1e-7, 1-1e-7)

        if len(y_true.shape) == 1: ##scalar 
            correct_confidences = y_pred_clipped[range(samples), y_true]
        elif len(y_true) == 2: ##one hot encoded
            correct_confidences = np.sum(y_pred_clipped*y_true, axis = 1)

        neg_log_likelihoods = -np.log(correct_confidences)
        return neg_log_likelihoods
    
    def backward(self, dvalues, y_true):
        samples = len(dvalues)
        labels = len(dvalues[0])

        if len(y_true.shape) == 1:
            y_true = np.eye(labels)[y_true]

        self.dinputs = -y_true/dvalues
        self.dinputs = self.dinputs/samples

In [363]:
'''Completely taken from the book'''
#Don't really see the point in it but it's nice to have

class Activation_Softmax_Loss_CategoricalCrossentropy():
# Creates activation and loss function objects
    def __init__(self):
        self.activation = Activation_Softmax()
        self.loss = CrossEntropy()
# Forward pass
    def forward(self, inputs, y_true):
# Output layer's activation function
        self.activation.forward(inputs)
        # Set the output
        self.output = self.activation.output
# Calculate and return loss value
        return self.loss.calculate(self.output, y_true)
    
    def backward(self, dvalues, y_true):
# Number of samples
        samples = len(dvalues)
        # If labels are one-hot encoded,
        # turn them into discrete values
        if len(y_true.shape) == 2:
            y_true = np.argmax(y_true, axis=1)
        # Copy so we can safely modify
        self.dinputs = dvalues.copy()
        # Calculate gradient
        self.dinputs[range(samples), y_true] -= 1
        # Normalize gradient
        self.dinputs = self.dinputs / samples

In [364]:
##learning rate to be updated after doing cross-validation(anywhere from 0,0001 to 1)


class Optimizer_SGD:
    def __init__(self, learning_rate = .85):
        self.learning_rate = learning_rate


    def update_params(self, layer):
        layer.weights += - self.learning_rate *layer.dweights
        layer.biases +=  -self.learning_rate * layer.dbiases


In [365]:
X, y = spiral_data(samples=100, classes=3)

In [366]:

dense1 = Layer_Dense(2, 64)
activation1 = Activation_ReLu()
dense2 = Layer_Dense(64, 3)
loss_activation = Activation_Softmax_Loss_CategoricalCrossentropy()

In [367]:
optimizer = Optimizer_SGD()

dense1.forward(X)

activation1.forward(dense1.output)

dense2.forward(activation1.output)

loss = loss_activation.forward(dense2.output, y)

print("Loss:", loss)

Loss: 1.0985352369055663


In [368]:
predictions = np.argmax(loss_activation.output, axis=1)
if len(y.shape) == 2:
    y = np.argmax(y, axis=1)
accuracy = np.mean(predictions==y)
print('acc:', accuracy) 

acc: 0.3333333333333333


In [369]:
loss_activation.backward(loss_activation.output, y)
dense2.backward(loss_activation.dinputs)
activation1.backward(dense2.dinputs)
dense1.backward(activation1.dinputs)

In [370]:
optimizer.update_params(dense1)
optimizer.update_params(dense2)

In [371]:
'''from book'''

for epoch in range(10001):
# Perform a forward pass of our training data through this layer
    dense1.forward(X)
    # Perform a forward pass through activation function
    # takes the output of first dense layer here
    activation1.forward(dense1.output)
    # Perform a forward pass through second Dense layer
    # takes outputs of activation function of first layer as inputs
    dense2.forward(activation1.output)
    # Perform a forward pass through the activation/loss function
    # takes the output of second dense layer here and returns loss
    loss = loss_activation.forward(dense2.output, y)

    predictions = np.argmax(loss_activation.output, axis=1)

    if len(y.shape) == 2:
         y = np.argmax(y, axis=1)
    accuracy = np.mean(predictions==y)

    if not epoch % 100:
         print(f'epoch: {epoch}, ' + 
                f'acc: {accuracy:.3f}, ' +
                f'loss: {loss:.3f}')
    # Backward pass
    loss_activation.backward(loss_activation.output, y)
    dense2.backward(loss_activation.dinputs)
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)
    # Update weights and biases
    optimizer.update_params(dense1)
    optimizer.update_params(dense2)

epoch: 0, acc: 0.387, loss: 1.096
epoch: 100, acc: 0.430, loss: 1.057
epoch: 200, acc: 0.463, loss: 1.053
epoch: 300, acc: 0.470, loss: 1.052
epoch: 400, acc: 0.470, loss: 1.050
epoch: 500, acc: 0.477, loss: 1.049
epoch: 600, acc: 0.480, loss: 1.047
epoch: 700, acc: 0.487, loss: 1.043
epoch: 800, acc: 0.490, loss: 1.037
epoch: 900, acc: 0.493, loss: 1.027
epoch: 1000, acc: 0.500, loss: 1.013
epoch: 1100, acc: 0.513, loss: 0.999
epoch: 1200, acc: 0.550, loss: 0.983
epoch: 1300, acc: 0.560, loss: 0.968
epoch: 1400, acc: 0.547, loss: 0.954
epoch: 1500, acc: 0.507, loss: 0.960
epoch: 1600, acc: 0.497, loss: 0.954
epoch: 1700, acc: 0.510, loss: 0.948
epoch: 1800, acc: 0.523, loss: 0.939
epoch: 1900, acc: 0.573, loss: 0.930
epoch: 2000, acc: 0.537, loss: 0.936
epoch: 2100, acc: 0.603, loss: 0.921
epoch: 2200, acc: 0.517, loss: 0.923
epoch: 2300, acc: 0.550, loss: 0.931
epoch: 2400, acc: 0.560, loss: 0.894
epoch: 2500, acc: 0.533, loss: 0.907
epoch: 2600, acc: 0.540, loss: 0.865
epoch: 2700, 

In [ ]:
"""
It's overfitting

I need to add regularization

Optiization of learning rate and the calculate the evaluation metrics, and it seems done

"""